In [1]:
import sys
sys.path.append("..")
from collections import Counter
from utils.data_utils import *
from tqdm import tqdm

In [29]:
data_dir = "datasets/Project SSQS Fixed Camera/val"
ids = []

viz_limits = {'fender_cover': 25, 'wheelrim': 25, 'lifting_pads':25}


annot_to_visualize = "lifting_pads"
padscount_with_cover = 0
for folderid in tqdm(os.listdir(data_dir)):
    if folderid.endswith("Store"):continue
    folder_path = f"{data_dir}/{folderid}"
    
    annot_file = read_json(f"{folder_path}/via_region_data.json")
    viz_count = {'fender_cover': 0, 'wheelrim':0, 'lifting_pads': 0}
    for imgname, annot in annot_file.items():
        imgpath = f"{folder_path}/{imgname}"
        img = read_img(imgpath)
        current_classes = []
        for region in annot['regions']:
            coords, identity = get_coords_from_annot(region)
                       
            bbox = get_bbox_from_polycoords(coords)
            
            img = draw_box(img, bbox, (255,0,0))
            img = write_text(img, identity, bbox[:2])
        
            current_classes.append(identity)

            ids.append(identity)
            
        viz_dir = f"datasets/viz_annots/all_viz"
        os.makedirs(viz_dir, exist_ok=True)
        #Image.fromarray(img).save(f"{viz_dir}/{imgname}")
        
        
Counter(ids)
            
            


100%|██████████| 4/4 [00:08<00:00,  2.12s/it]


Counter({'fender_cover': 29, 'wheelrim': 162, 'lifting_pads': 728})

In [1]:
# train class distribution: {'wheelrim': 530, 'lifting_pads': 731, 'fender_cover': 277}
# val class distribution: {'fender_cover': 29, 'wheelrim': 162, 'lifting_pads': 728}

sorted(['fender_cover', 'wheelrim', 'lifting_pads'])

['fender_cover', 'lifting_pads', 'wheelrim']

## Filter out lifting pads that are annotated on fender views

In [22]:

both_imgpaths = [] #imgpaths where both liftingpads and fender/fendercover is present.

data_dir = "Project SSQS Fixed Camera"


ids = []
c = 0
for folderid in os.listdir(data_dir):
    if folderid.endswith("Store"):continue
    folder_path = f"{data_dir}/{folderid}"
    
    annot_file = read_json(f"{folder_path}/via_region_data.json")
    
    for imgname, annot in tqdm(annot_file.items()):
        imgpath = f"{folder_path}/{imgname}"
        img = read_img(imgpath)
        
        
        current_classes = []
        for region in annot['regions']:
            coords, identity = get_coords_from_annot(region)
            current_classes.append(identity)
            ids.append(identity)
            bbox = get_bbox_from_polycoords(coords)
            img = draw_box(img, bbox, (255,0,0))
            img = write_text(img, identity, bbox[:2])

    
            
        if ('fender_cover' in current_classes and 'lifting_pads' in current_classes):
            #Image.fromarray(img).save(f"viz_annots/random/{imgname}")
            c += 1
            
        #elif 'lifting_pads' in current_classes and 'wheelrim' not in current_classes:
        #    Image.fromarray(img).save(f"viz_annots/only_lifting_pads/{imgname}") #front-view needs to be filtered after this step.
        #    pass
            


c

100%|██████████| 200/200 [00:03<00:00, 66.49it/s]


217

In [21]:
#Updated annot: {'fender_cover': 306, 'wheelrim': 692, 'lifting_pads': 1459}
#Unfiltered annot: {'fender_cover': 306, 'wheelrim': 692, 'lifting_pads': 1867}

Counter({'fender_cover': 306, 'wheelrim': 692, 'lifting_pads': 1459})

408

In [18]:
# Create new annotation leaving out lifting pads annotation from camera3 view
data_dir = "Project SSQS Fixed Camera"

c = 0
for folderid in os.listdir(data_dir):
    if folderid.endswith("Store"):continue
    folder_path = f"{data_dir}/{folderid}"
    
    annot_file = read_json(f"{folder_path}/via_region_data.json")
    
    for imgname, annot in tqdm(annot_file.items()):
        new_region = []

        current_classes = []
        for region in annot['regions']:
            coords, identity = get_coords_from_annot(region)
            
            #Dont add caddy-view lifting pads
            if imgname in os.listdir("viz_annots/images_with_both_liftingpad_and_fender_annot") and identity == 'lifting_pads':
                continue
            
            new_region.append(region)
            
            
        annot_file[imgname]['regions'] = new_region
        
        
        
    #dump_json(annot_file, f"{folder_path}/via_region_data2.json", indent = 1)

100%|██████████| 200/200 [00:00<00:00, 3274.05it/s]


In [26]:
# Delete via_region_data and rename via_region_data2 to via_region_data
data_dir = "Project SSQS Fixed Camera"

c = 0
for folderid in os.listdir(data_dir):
    if folderid.endswith("Store"):continue
    folder_path = f"{data_dir}/{folderid}"
    #os.rename(f"{folder_path}/via_region_data.json", f"{folder_path}/via_region_data_old.json")
    os.rename(f"{folder_path}/via_region_data2.json", f"{folder_path}/via_region_data.json")